<a href="https://colab.research.google.com/github/tylerb55/COMP530/blob/main/ImageNetPretrainedMulticlass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https://github.com/tylerb55/COMP530.git

Cloning into 'COMP530'...
remote: Enumerating objects: 4276, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 4276 (delta 33), reused 0 (delta 0), pack-reused 4216
Receiving objects: 100% (4276/4276), 917.13 MiB | 16.29 MiB/s, done.
Resolving deltas: 100% (114/114), done.
Checking out files: 100% (1162/1162), done.


In [ ]:
import numpy as np
import pandas as pd
import collections
import matplotlib.image as img
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import PIL.Image as Image

# **Split the train test and validation set**

In [ ]:
def Train_Validation_Test_Set(directory_name):
  """a function to load the images in a large directory into a tensorflow dataset object
  the data is split 80:10:10 in training:validation:test. The dataset is shuffled before splitting
  and the images are formatted to 512x512 pixels (pixel values range from 0 to 255)
  Images are labelled 0,1 based on the folder they are in. Normalcases are 0 and 
  cancercases are 1"""
  dataset_train=tf.keras.preprocessing.image_dataset_from_directory('/content/COMP530/'+directory_name,
                                                                    labels='inferred',
                                                                    label_mode='int',
                                                                    class_names=['type a','type b','type e','type g'],
                                                                    image_size=(512,512),
                                                                    shuffle=True,
                                                                    seed=305,
                                                                    validation_split=0.2,
                                                                    subset='training',
                                                                    batch_size=None
                                                                    )

  dataset_validation=tf.keras.preprocessing.image_dataset_from_directory('/content/COMP530/'+directory_name,
                                                                    labels='inferred',
                                                                    label_mode='int',
                                                                    class_names=['type a','type b','type e','type g'],
                                                                    image_size=(512,512),
                                                                    shuffle=True,
                                                                    seed=305,
                                                                    validation_split=0.2,
                                                                    subset='validation',
                                                                    batch_size=None
                                                                    )
  
  dataset_test=tf.keras.preprocessing.image_dataset_from_directory('/content/COMP530/'+directory_name,
                                                                    labels='inferred',
                                                                    label_mode='int',
                                                                    class_names=['type a','type b','type e','type g'],
                                                                    image_size=(512,512),
                                                                    shuffle=True,
                                                                    seed=305,
                                                                    validation_split=0.1,
                                                                    subset='validation',
                                                                    batch_size=None
                                                                    )
    
  dataset_validation=dataset_validation.take(dataset_test.__len__())

  return dataset_train,dataset_validation,dataset_test

In [ ]:
def Preprocess(image,label):
  image = image/255.0
  # Resize images from 512x512 to 224x224
  image = tf.image.resize(image, (224,224))
  return image, label

In [ ]:
BATCH_SIZE=32

dataset_train,dataset_validation, dataset_test=Train_Validation_Test_Set("Dataset2")
dataset_train=dataset_train.map(Preprocess).batch(batch_size=BATCH_SIZE,drop_remainder=True)
dataset_validation=dataset_validation.map(Preprocess).batch(batch_size=BATCH_SIZE,drop_remainder=True)
dataset_test=dataset_test.map(Preprocess).batch(batch_size=BATCH_SIZE,drop_remainder=True)

AUTOTUNE = tf.data.AUTOTUNE
dataset_train = dataset_train.cache().prefetch(buffer_size=AUTOTUNE)
dataset_validation = dataset_validation.cache().prefetch(buffer_size=AUTOTUNE)

Found 1097 files belonging to 2 classes.
Using 878 files for training.
Found 1097 files belonging to 2 classes.
Using 219 files for validation.
Found 1097 files belonging to 2 classes.
Using 109 files for validation.


# **MobileNetV2**

In [ ]:
base_model=tf.keras.applications.MobileNetV2(weights='imagenet',input_shape=(224,224,3),include_top=False) # use mobilenetv2 as the base for the tl model
base_model.trainable = False # freeze the mobilenetv2 layers

inputs = tf.keras.Input(shape=(224, 224, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`.
x = base_model(inputs, training=True)
# Convert features of shape `base_model.output_shape[1:]` to vectors
#x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128,activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
# A Dense classifier with a 4 unit (categorical classification)
outputs = tf.keras.layers.Dense(4,activation='softmax')(x)
mobilenet = tf.keras.Model(inputs, outputs)

mobilenet.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d_1   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_1 (Dropout)         (None, 1280)              0         
                                                                 
 dense_3 (Dense)             (None, 1)                 1281      
                                                                 
Total params: 2,259,265
Trainable params: 1,281
Non-trainab

In [ ]:
mobilenet.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.BinaryCrossentropy(),
  metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.TruePositives(),tf.keras.metrics.FalsePositives(),tf.keras.metrics.TrueNegatives(),tf.keras.metrics.FalseNegatives()])

NUM_EPOCHS = 30

history = mobilenet.fit(dataset_train,
                    validation_data=dataset_validation,
                    epochs=NUM_EPOCHS)

Epoch 1/30
27/27 [==============================] - 26s 837ms/step - loss: 0.5812 - binary_accuracy: 0.6852 - true_positives_1: 428.0000 - false_positives_1: 163.0000 - true_negatives_1: 164.0000 - false_negatives_1: 109.0000
Epoch 2/30
27/27 [==============================] - 22s 828ms/step - loss: 0.3631 - binary_accuracy: 0.8414 - true_positives_1: 473.0000 - false_positives_1: 73.0000 - true_negatives_1: 254.0000 - false_negatives_1: 64.0000
Epoch 3/30
27/27 [==============================] - 22s 830ms/step - loss: 0.3306 - binary_accuracy: 0.8576 - true_positives_1: 470.0000 - false_positives_1: 56.0000 - true_negatives_1: 271.0000 - false_negatives_1: 67.0000
Epoch 4/30
27/27 [==============================] - 22s 831ms/step - loss: 0.3092 - binary_accuracy: 0.8762 - true_positives_1: 476.0000 - false_positives_1: 46.0000 - true_negatives_1: 281.0000 - false_negatives_1: 61.0000
Epoch 5/30
27/27 [==============================] - 22s 832ms/step - loss: 0.2739 - binary_accuracy: 0

KeyboardInterrupt: ignored

In [ ]:
# summarize history for accuracy
plt.plot(history.history['binary_accuracy'])
plt.plot(history.history['val_binary_accuracy'])
plt.title('training accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('training loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

KeyError: ignored

# **ResNet50**

In [ ]:
base_model=tf.keras.applications.ResNet50(weights='imagenet',input_shape=(224,224,3),include_top=False) # use resnet50 as the base for the tl model
base_model.trainable = False # freeze the resnet50 layers

inputs = tf.keras.Input(shape=(224, 224, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`.
x = base_model(inputs, training=True)
# Convert features of shape `base_model.output_shape[1:]` to vectors
#x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128,activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
# A Dense classifier with a 4 unit (categorical classification)
outputs = tf.keras.layers.Dense(4,activation='softmax')(x)
resnet = tf.keras.Model(inputs, outputs)

resnet.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 2048)              23561152  
                                                                 
 dense_3 (Dense)             (None, 1)                 2049      
                                                                 
Total params: 23,563,201
Trainable params: 2,049
Non-trainable params: 23,561,152
_________________________________________________________________


In [ ]:
resnet.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.BinaryCrossentropy(),
  metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.TruePositives(),tf.keras.metrics.FalsePositives(),tf.keras.metrics.TrueNegatives(),tf.keras.metrics.FalseNegatives()])

NUM_EPOCHS = 30

history = resnet.fit(dataset_train,
                    validation_data=dataset_validation,
                    epochs=NUM_EPOCHS)

Epoch 1/10
27/27 [==============================] - 9s 105ms/step - loss: 0.4551 - binary_accuracy: 0.7650 - precision_1: 0.8047 - val_loss: 0.4348 - val_binary_accuracy: 0.8333 - val_precision_1: 0.8387
Epoch 2/10
27/27 [==============================] - 2s 68ms/step - loss: 0.3064 - binary_accuracy: 0.8773 - precision_1: 0.9168 - val_loss: 0.3896 - val_binary_accuracy: 0.8542 - val_precision_1: 0.8667
Epoch 3/10
27/27 [==============================] - 2s 66ms/step - loss: 0.2700 - binary_accuracy: 0.8900 - precision_1: 0.9438 - val_loss: 0.3471 - val_binary_accuracy: 0.8333 - val_precision_1: 0.8621
Epoch 4/10
27/27 [==============================] - 2s 66ms/step - loss: 0.2452 - binary_accuracy: 0.8981 - precision_1: 0.9517 - val_loss: 0.3243 - val_binary_accuracy: 0.8542 - val_precision_1: 0.8793
Epoch 5/10
27/27 [==============================] - 2s 66ms/step - loss: 0.2264 - binary_accuracy: 0.8970 - precision_1: 0.9534 - val_loss: 0.3080 - val_binary_accuracy: 0.8542 - val_prec

In [ ]:
# summarize history for accuracy
plt.plot(history.history['binary_accuracy'])
plt.plot(history.history['val_binary_accuracy'])
plt.title('training accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('training loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# **DenseNet121**

In [ ]:
base_model=tf.keras.applications.DenseNet121(weights='imagenet',input_shape=(224,224,3),include_top=False) # use densenet121 as the base for the tl model
base_model.trainable = False # freeze the densenet121 layers

inputs = tf.keras.Input(shape=(224, 224, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`.
x = base_model(inputs, training=True)
# Convert features of shape `base_model.output_shape[1:]` to vectors
#x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128,activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
# A Dense classifier with a 4 unit (categorical classification)
outputs = tf.keras.layers.Dense(4,activation='softmax')(x)
densenet = tf.keras.Model(inputs, outputs)

densenet.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 2048)              23561152  
                                                                 
 dense_3 (Dense)             (None, 1)                 2049      
                                                                 
Total params: 23,563,201
Trainable params: 2,049
Non-trainable params: 23,561,152
_________________________________________________________________


In [ ]:
densenet.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.BinaryCrossentropy(),
  metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.TruePositives(),tf.keras.metrics.FalsePositives(),tf.keras.metrics.TrueNegatives(),tf.keras.metrics.FalseNegatives()])

NUM_EPOCHS = 30

history = densenet.fit(dataset_train,
                    validation_data=dataset_validation,
                    epochs=NUM_EPOCHS)

Epoch 1/10
27/27 [==============================] - 9s 105ms/step - loss: 0.4551 - binary_accuracy: 0.7650 - precision_1: 0.8047 - val_loss: 0.4348 - val_binary_accuracy: 0.8333 - val_precision_1: 0.8387
Epoch 2/10
27/27 [==============================] - 2s 68ms/step - loss: 0.3064 - binary_accuracy: 0.8773 - precision_1: 0.9168 - val_loss: 0.3896 - val_binary_accuracy: 0.8542 - val_precision_1: 0.8667
Epoch 3/10
27/27 [==============================] - 2s 66ms/step - loss: 0.2700 - binary_accuracy: 0.8900 - precision_1: 0.9438 - val_loss: 0.3471 - val_binary_accuracy: 0.8333 - val_precision_1: 0.8621
Epoch 4/10
27/27 [==============================] - 2s 66ms/step - loss: 0.2452 - binary_accuracy: 0.8981 - precision_1: 0.9517 - val_loss: 0.3243 - val_binary_accuracy: 0.8542 - val_precision_1: 0.8793
Epoch 5/10
27/27 [==============================] - 2s 66ms/step - loss: 0.2264 - binary_accuracy: 0.8970 - precision_1: 0.9534 - val_loss: 0.3080 - val_binary_accuracy: 0.8542 - val_prec

In [ ]:
# summarize history for accuracy
plt.plot(history.history['binary_accuracy'])
plt.plot(history.history['val_binary_accuracy'])
plt.title('training accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('training loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# **VGG16**

In [ ]:
base_model=tf.keras.applications.VGG16(weights='imagenet',input_shape=(224,224,3),include_top=False) # use vgg16 as the base for the tl model
base_model.trainable = False # freeze the vgg16 layers

inputs = tf.keras.Input(shape=(224, 224, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`.
x = base_model(inputs, training=True)
# Convert features of shape `base_model.output_shape[1:]` to vectors
#x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128,activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
# A Dense classifier with a 4 unit (categorical classification)
outputs = tf.keras.layers.Dense(4,activation='softmax')(x)
vgg = tf.keras.Model(inputs, outputs)

vgg.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 2048)              23561152  
                                                                 
 dense_3 (Dense)             (None, 1)                 2049      
                                                                 
Total params: 23,563,201
Trainable params: 2,049
Non-trainable params: 23,561,152
_________________________________________________________________


In [ ]:
vgg.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.BinaryCrossentropy(),
  metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.TruePositives(),tf.keras.metrics.FalsePositives(),tf.keras.metrics.TrueNegatives(),tf.keras.metrics.FalseNegatives()])

NUM_EPOCHS = 30

history = vgg.fit(dataset_train,
                    validation_data=dataset_validation,
                    epochs=NUM_EPOCHS)

Epoch 1/10
27/27 [==============================] - 9s 105ms/step - loss: 0.4551 - binary_accuracy: 0.7650 - precision_1: 0.8047 - val_loss: 0.4348 - val_binary_accuracy: 0.8333 - val_precision_1: 0.8387
Epoch 2/10
27/27 [==============================] - 2s 68ms/step - loss: 0.3064 - binary_accuracy: 0.8773 - precision_1: 0.9168 - val_loss: 0.3896 - val_binary_accuracy: 0.8542 - val_precision_1: 0.8667
Epoch 3/10
27/27 [==============================] - 2s 66ms/step - loss: 0.2700 - binary_accuracy: 0.8900 - precision_1: 0.9438 - val_loss: 0.3471 - val_binary_accuracy: 0.8333 - val_precision_1: 0.8621
Epoch 4/10
27/27 [==============================] - 2s 66ms/step - loss: 0.2452 - binary_accuracy: 0.8981 - precision_1: 0.9517 - val_loss: 0.3243 - val_binary_accuracy: 0.8542 - val_precision_1: 0.8793
Epoch 5/10
27/27 [==============================] - 2s 66ms/step - loss: 0.2264 - binary_accuracy: 0.8970 - precision_1: 0.9534 - val_loss: 0.3080 - val_binary_accuracy: 0.8542 - val_prec

In [ ]:
# summarize history for accuracy
plt.plot(history.history['binary_accuracy'])
plt.plot(history.history['val_binary_accuracy'])
plt.title('training accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('training loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# **InceptionV3**

In [ ]:
base_model=tf.keras.applications.InceptionV3(weights='imagenet',input_shape=(224,224,3),include_top=False) # use inceptionv3 as the base for the tl model
base_model.trainable = False # freeze the inceptionv3 layers

inputs = tf.keras.Input(shape=(224, 224, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`.
x = base_model(inputs, training=True)
# Convert features of shape `base_model.output_shape[1:]` to vectors
#x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128,activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
# A Dense classifier with a 4 unit (categorical classification)
outputs = tf.keras.layers.Dense(4,activation='softmax')(x)
inception = tf.keras.Model(inputs, outputs)

inception.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 2048)              23561152  
                                                                 
 dense_3 (Dense)             (None, 1)                 2049      
                                                                 
Total params: 23,563,201
Trainable params: 2,049
Non-trainable params: 23,561,152
_________________________________________________________________


In [ ]:
inception.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.BinaryCrossentropy(),
  metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.TruePositives(),tf.keras.metrics.FalsePositives(),tf.keras.metrics.TrueNegatives(),tf.keras.metrics.FalseNegatives()])

NUM_EPOCHS = 30

history = inception.fit(dataset_train,
                    validation_data=dataset_validation,
                    epochs=NUM_EPOCHS)

Epoch 1/10
27/27 [==============================] - 9s 105ms/step - loss: 0.4551 - binary_accuracy: 0.7650 - precision_1: 0.8047 - val_loss: 0.4348 - val_binary_accuracy: 0.8333 - val_precision_1: 0.8387
Epoch 2/10
27/27 [==============================] - 2s 68ms/step - loss: 0.3064 - binary_accuracy: 0.8773 - precision_1: 0.9168 - val_loss: 0.3896 - val_binary_accuracy: 0.8542 - val_precision_1: 0.8667
Epoch 3/10
27/27 [==============================] - 2s 66ms/step - loss: 0.2700 - binary_accuracy: 0.8900 - precision_1: 0.9438 - val_loss: 0.3471 - val_binary_accuracy: 0.8333 - val_precision_1: 0.8621
Epoch 4/10
27/27 [==============================] - 2s 66ms/step - loss: 0.2452 - binary_accuracy: 0.8981 - precision_1: 0.9517 - val_loss: 0.3243 - val_binary_accuracy: 0.8542 - val_precision_1: 0.8793
Epoch 5/10
27/27 [==============================] - 2s 66ms/step - loss: 0.2264 - binary_accuracy: 0.8970 - precision_1: 0.9534 - val_loss: 0.3080 - val_binary_accuracy: 0.8542 - val_prec

In [ ]:
# summarize history for accuracy
plt.plot(history.history['binary_accuracy'])
plt.plot(history.history['val_binary_accuracy'])
plt.title('training accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('training loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# **Evaluation**

In [ ]:
mobilenet.evaluate(dataset_test)

In [ ]:
resnet.evaluate(dataset_test)

In [ ]:
densenet.evaluate(dataset_test)

In [ ]:
vgg.evaluate(dataset_test)

In [ ]:
inception.evaluate(dataset_test)